# Project 4 

Outline:

* scraping data

* binning: by salary

* parsing data: tokenizing, bigrams, most_common words for each bin 

* building model

* evaluating model


Downloading the whole data for US. 09/29/2016

In [127]:
df

,company,job_link,job_location,job_posted,job_title
0,TIAA,http://www.indeed.com/rc/clk?jk=c13a0f1f2c8f3bcd&fccid=ee5ba787b9f4e31a,"New York, NY 10002 (Lower East Side area)",Just posted,"Data Scientist, Fraud Data Analytics, Enterprise Counter-Fraud"
1,Huawei,http://www.indeed.com/rc/clk?jk=20589484a63ab6af&fccid=d8be85d73018fac8,"Santa Clara, CA",Just posted,Data Scientist
2,Proofpoint,http://www.indeed.com/rc/clk?jk=88441dea622e8a17&fccid=69c7d55b78dc7424,"Sunnyvale, CA 94089",Just posted,Data Scientist
3,Mobile Defense Inc.,http://www.indeed.com/rc/clk?jk=8ddae5deb67e7871&fccid=90b9bb2a9e190547,"Cleveland, OH",Just posted,Data Scientist
4,Inficare,http://www.indeed.com/rc/clk?jk=dd28a2576d36c5e3&fccid=35781c2041de2a91,"New York, NY",Just posted,Data Scientist
5,Continental Technology Solutions,http://www.indeed.com/rc/clk?jk=aab733e36c65e6a7&fccid=664cda5b7c62a987,"San Diego, CA",Just posted,Data Scientist
6,"Cisco Systems, Inc.",http://www.indeed.com/rc/clk?jk=6cb402d057d00c35&fccid=dfc44f3b8c44a6db,"San Jose, CA",Just posted,Data Scientist - Technology and Quality
7,Goodway Group,http://www.indeed.com/rc/clk?jk=3910d2146bc680c1&fccid=920e697cdf154a3c,United States,Just posted,Data Scientist - Virtual
8,The Nielsen Company,http://www.indeed.com/rc/clk?jk=56aea398a23ae3e4&fccid=61bb2b2cdd4507ae,"San Francisco, CA",Just posted,Lead Data Scientist
9,Axius Technologies,http://www.indeed.com/rc/clk?jk=efee1987084eb21b&fccid=e5ea8513745bad23,"Plano, TX",Just posted,Data Scientist


In [128]:
# check one entry to verify contents
print df.iloc[6]

company                                                             Cisco Systems, Inc.
job_link        http://www.indeed.com/rc/clk?jk=6cb402d057d00c35&fccid=dfc44f3b8c44a6db
job_location                                                               San Jose, CA
job_posted                                                                  Just posted
job_title                                       Data Scientist - Technology and Quality
Name: 6, dtype: object


In [129]:
# Save the result to CSV
df.to_csv('./indeed_companies.csv', encoding='utf-8')

Downloading the whole data for Australia. 09/29/2016

In [130]:
# load the library


df_au = pd.DataFrame()   # create a new data frame
pd.set_option('max_colwidth',500)    # to remove column limit (default is 50. Otherwise, we'll lose some info)


# indeed.com's query url
base_url = 'http://au.indeed.com/jobs?q=Data+Scientist&jt=fulltime'
sort_by = 'date'          # sort by data
start_from = '&start='    # start page number

In [131]:
for page in range(1,10): # page from 1 to 100 
    page = (page-1) * 10   # pagination
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
    contents = BeautifulSoup(urllib2.urlopen(url), "lxml") 

    contentsElements_au = contents.find_all('div', attrs={"class" : '  row  result'})
    cont_au = contents.find('div', attrs = {'class': 'lastRow  row  result'}) ##just added last job on each page
    contentsElements_au.append(cont_au)
    # each row contains 1 job

    # extract job information
    for elem in contentsElements_au: 
        try:
            company_au = elem.find('span', attrs={'itemprop':'name'}).getText().strip() #strips off whitespace 
        except AttributeError:
            company = '-'
        job_title_au = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        home_url_au = "http://www.indeed.com"
        job_link_au = "%s%s" % (home_url,elem.find('a').get('href'))
        job_addr_au = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()
        job_posted_au = elem.find('span', attrs={'class': 'date'}).getText()

           # add a record of job info to our data frame

        df_au = df_au.append({'company': company_au, 
                        'job_title': job_title_au, 
                        'job_link': job_link_au, 
                        'job_posted': job_posted_au,
                        'job_location': job_addr_au
                       }, ignore_index=True)
df_au

,company,job_link,job_location,job_posted,job_title
0,Virgin Australia,http://www.indeed.com/rc/clk?jk=e878a8a83d2ae2a2&fccid=6fc61b1e19b99539,Sydney NSW,6 days ago,Data Scientist
1,AGL Energy,http://www.indeed.com/rc/clk?jk=0d01c297c64b07ea&fccid=04bdf85f0d53c42e,Melbourne City Centre VIC,8 days ago,Data Scientist
2,VicSuper,http://www.indeed.com/rc/clk?jk=d1006321ae59e520&fccid=a53fe96b2e366c58,Melbourne City Centre VIC,9 days ago,Data Scientist
3,General Assembly,http://www.indeed.com/rc/clk?jk=49481b391a9be1cc&fccid=a4cd0efa7775ecd2,Sydney NSW,30+ days ago,Part Time Data Science Instructor
4,CSIRO,http://www.indeed.com/rc/clk?jk=22027439f55f2876&fccid=f6e6c4d709d8e570,Sydney NSW,14 days ago,Postdoctoral Fellow - Data Analytics
5,Oneflare,http://www.indeed.com/rc/clk?jk=b21e63711d775880&fccid=524cc6038f9a26af,Sydney NSW,30+ days ago,Data Scientist
6,Atlassian,http://www.indeed.com/rc/clk?jk=b61dc9bc2976ccf0&fccid=e6d4ba9e2cfe7902,Sydney NSW,27 days ago,Data / Product Analyst
7,EY,http://www.indeed.com/rc/clk?jk=71d7092a5d3cae74&fccid=1544766d4c2915b0,Melbourne VIC,17 days ago,Data Scientist
8,CSIRO,http://www.indeed.com/rc/clk?jk=6bee32a45106f007&fccid=f6e6c4d709d8e570,Sydney NSW,1 day ago,CSIRO Postdoctoral Fellow: ASKAP Big Data Innovation Challenge
9,MerchMiner,http://www.indeed.com/rc/clk?jk=dea630d172d52806&fccid=a1b5a19ea0dca1b3,Rose Park SA,6 days ago,DATA SCIENTIST/ALGORITHM NINJA


In [132]:
# Save the result to CSV
df_au.to_csv('./indeed_companies_au.csv', encoding='utf-8')

In [29]:
jobs_data = pd.read_csv('indeed_companies.csv')
jobs_data_au = pd.read_csv('indeed_companies_au.csv')
frames = [jobs_data, jobs_data_au]
result = pd.concat(frames, axis = 0, ignore_index = True)
result

,Unnamed: 0,company,job_link,job_location,job_posted,job_title
0,0,TIAA,http://www.indeed.com/rc/clk?jk=c13a0f1f2c8f3bcd&fccid=ee5ba787b9f4e31a,"New York, NY 10002 (Lower East Side area)",Just posted,"Data Scientist, Fraud Data Analytics, Enterprise Counter-Fraud"
1,1,Huawei,http://www.indeed.com/rc/clk?jk=20589484a63ab6af&fccid=d8be85d73018fac8,"Santa Clara, CA",Just posted,Data Scientist
2,2,Proofpoint,http://www.indeed.com/rc/clk?jk=88441dea622e8a17&fccid=69c7d55b78dc7424,"Sunnyvale, CA 94089",Just posted,Data Scientist
3,3,Mobile Defense Inc.,http://www.indeed.com/rc/clk?jk=8ddae5deb67e7871&fccid=90b9bb2a9e190547,"Cleveland, OH",Just posted,Data Scientist
4,4,Inficare,http://www.indeed.com/rc/clk?jk=dd28a2576d36c5e3&fccid=35781c2041de2a91,"New York, NY",Just posted,Data Scientist
5,5,Continental Technology Solutions,http://www.indeed.com/rc/clk?jk=aab733e36c65e6a7&fccid=664cda5b7c62a987,"San Diego, CA",Just posted,Data Scientist
6,6,"Cisco Systems, Inc.",http://www.indeed.com/rc/clk?jk=6cb402d057d00c35&fccid=dfc44f3b8c44a6db,"San Jose, CA",Just posted,Data Scientist - Technology and Quality
7,7,Goodway Group,http://www.indeed.com/rc/clk?jk=3910d2146bc680c1&fccid=920e697cdf154a3c,United States,Just posted,Data Scientist - Virtual
8,8,The Nielsen Company,http://www.indeed.com/rc/clk?jk=56aea398a23ae3e4&fccid=61bb2b2cdd4507ae,"San Francisco, CA",Just posted,Lead Data Scientist
9,9,Axius Technologies,http://www.indeed.com/rc/clk?jk=efee1987084eb21b&fccid=e5ea8513745bad23,"Plano, TX",Just posted,Data Scientist


.

In [1]:
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''      

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()

    return None


def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])


In [197]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2, requests


url_template = "http://www.indeed.com/jobs?q=data+scientist+${}%2C000&l=USA&start={}"
max_results_per_city = 5000 # Set this to a high-value (5000) to generate more results. 

# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

#city = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()

rows = []
for salary in set([ '55-70', '70-85', '85-100', '100-115']):
    for start in range(1,100): 
        r = requests.get(url_template.format(salary, start))
        soup = BeautifulSoup(r.content) 
        results = soup.findAll('div', { "class" : "result" })
        for result in results:
            if result:
                row = {}
                row['title'] = get_title_from_result(result)
                row['company'] = get_company_from_result(result)
                row['summary'] = get_summary_from_result(result)
                row['city'] = get_location_from_result(result) 
                row['bin'] = salary
            #row['salary'] = get_salary_from_result(result)
                rows.append(row)
                

In [199]:
result = pd.DataFrame.from_records(rows)   # create a new data frame
result.drop_duplicates(subset = "summary")
result

,bin,city,company,summary,title
0,85-100,"Palo Alto, CA 94301 (Professorville area)",Verizon,Responsibilities Design test plans and test cases for machine learning and analytics based software products; Develop methodologies and tools to verify and,Data Scientist
1,85-100,"Chicago, IL",Amyx,"Amyx is seeking to hire a Data Scientist in Chicago, IL to support a large contract. Seeking a Senior Data Scientist with experience and knowledge of analytics...",Data Scientist
2,85-100,"Scottsdale, AZ",Vixxo,"As a member of the Product Development team, the Data Scientist will create, optimize and utilize multiple sets of data while applying strong data mining, data...",Data Scientist
3,85-100,"Redmond, WA 98052",Microsoft,"Knowledge of Hadoop, or other big data systems. We frequently partner with teams outside of Office in order to leverage the latest NLP technologies and data...",DATA & APPLIED SCIENTIST 2
4,85-100,"Minneapolis, MN 55426",General Mills,General Mills is seeking a Data Scientist to join a new data science team in the Global Business Solutions (shared services organization) that is tasked with...,Data Scientist
5,85-100,"Irvine, CA",Blizzard Entertainment,"Senior data engineers, data scientists, business analysts, and gameplay designers and engineers. To build out his or her professional foundation with the...","Data Engineer Intern, Business Intelligence"
6,85-100,"Irvine, CA",Blizzard Entertainment,Blizzard is looking for a data scientist to join our business intelligence team. Experience with data visualization tools like Tableau....,"Data Scientist, Business Intelligence"
7,85-100,Home Based,Dell,"Solid background in Mathematics, Statistics, and Data Mining. Intimacy dealing with data preparation, integration, transformation, and cleansing....",Advisory Data Scientist
8,85-100,"Phoenix, AZ",The Red Flag Group,"Experience in data prediction. Perform complex research data analysis. Assist in data cleansing and mining. Due to our rapid expansion and new product launch,...",Data Scientist
9,85-100,"Tampa, FL",ISPA Technology,"Seeking a Data Scientist to support the Intelligence Community and at Special Operations Command. Ability to merge data sources together, ensure consistency of...",Data Scientist


.

In [200]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 5 columns):
bin        3960 non-null object
city       3960 non-null object
company    3960 non-null object
summary    3960 non-null object
title      3960 non-null object
dtypes: object(5)
memory usage: 154.8+ KB


In [201]:
%matplotlib inline
import seaborn
result.bin.value_counts()

85-100     990
70-85      990
55-70      990
100-115    990
Name: bin, dtype: int64

In [223]:
#result.company.unique()

In [224]:
import nltk
import nltk.stem
import nltk.stem.porter
import nltk.stem.snowball
snowball = nltk.stem.snowball.SnowballStemmer("english")
from nltk.corpus import stopwords
import re

Extracting data from DataFrame and Cleaning it from numbers, charactters, stopwords.

In [225]:
junior = str(result[result['bin'].isin(['55-70'])].summary) #subsetting over categories(bins)
mid_low = str(result[result['bin'].isin(['70-85'])].summary)
mid_high = str(result[result['bin'].isin(['85-100'])].summary)
senior = str(result[result['bin'].isin(['100-115'])].summary)

junior_clean = re.sub("[^a-zA-Z]", " ", junior)
mid_low_clean = re.sub("[^a-zA-Z]", " ", mid_low)
mid_high_clean = re.sub("[^a-zA-Z]", " ", mid_high)
senior_clean = re.sub("[^a-zA-Z]", " ", senior)

#junior_clean

stop = set(stopwords.words('english'))
stop.update([',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '...'])

junior_words = [w for w in (nltk.word_tokenize(junior_clean)) if w.lower() not in stop]
mid_low_words = [w for w in (nltk.word_tokenize(mid_low_clean)) if w.lower() not in stop]
mid_high_words = [w for w in (nltk.word_tokenize(mid_high_clean)) if w.lower() not in stop]
senior_words = [w for w in (nltk.word_tokenize(senior_clean)) if w.lower() not in stop]


len(senior_words)

907

In [226]:
from nltk.probability import FreqDist
fdist_junior = FreqDist(junior_words)
jun_features = sorted(w for w in set(junior_words) if len(w) > 2 and fdist_junior[w] > 5)
jun_features

['Ability',
 'Data',
 'Houston',
 'Institute',
 'Investigate',
 'Research',
 'Science',
 'Statistics',
 'Support',
 'Technical',
 'analyses',
 'analysis',
 'analyzing',
 'data',
 'experience',
 'extracting',
 'identify',
 'interpreting',
 'investigating',
 'large',
 'love',
 'mining',
 'modeling',
 'models',
 'parsing',
 'patterns',
 'perform',
 'person',
 'position',
 'problems',
 'research',
 'science',
 'scientific',
 'seeking',
 'segmentation',
 'staff',
 'work',
 'working']

In [227]:
from nltk.probability import FreqDist
fdist_mid_low = FreqDist(mid_low_words)
sorted(w for w in set(mid_low_words) if len(w) > 2 and fdist_mid_low[w] > 5)

['Ability',
 'Data',
 'Develop',
 'Division',
 'Experience',
 'Physics',
 'Scientist',
 'Staff',
 'Systems',
 'analysis',
 'analyze',
 'based',
 'candidate',
 'clinical',
 'computer',
 'data',
 'ensure',
 'experience',
 'focused',
 'health',
 'interpret',
 'large',
 'mining',
 'new',
 'quality',
 'science',
 'sets',
 'sources',
 'statistical',
 'successful',
 'team',
 'using',
 'working']

In [228]:
from nltk.probability import FreqDist
fdist_mid_high = FreqDist(mid_high_words)
sorted(w for w in set(mid_high_words) if len(w) > 2 and fdist_mid_high[w] > 5)

['Data',
 'Experience',
 'Operations',
 'Scientist',
 'Seeking',
 'Senior',
 'analysis',
 'analytics',
 'applying',
 'build',
 'business',
 'cleansing',
 'data',
 'database',
 'engineers',
 'experience',
 'join',
 'large',
 'learning',
 'machine',
 'mining',
 'modeling',
 'models',
 'new',
 'quality',
 'seeking',
 'sources',
 'statistical',
 'support',
 'team',
 'test',
 'tools']

In [229]:
from nltk.probability import FreqDist
fdist_senior = FreqDist(senior_words)
sorted(w for w in set(senior_words) if len(w) > 2 and fdist_senior[w] > 5)

['Analysis',
 'Analytics',
 'Data',
 'Experience',
 'Field',
 'Oculus',
 'Proven',
 'Research',
 'Science',
 'Scientist',
 'Scientists',
 'analytics',
 'candidates',
 'data',
 'driven',
 'end',
 'engineers',
 'experience',
 'highly',
 'knowledge',
 'large',
 'leading',
 'looking',
 'management',
 'mining',
 'motivated',
 'multi',
 'position',
 'scientist',
 'scientists',
 'seeking',
 'skills',
 'software',
 'team',
 'test',
 'tools',
 'work',
 'working',
 'years']

Now lets have a look at bigrams and trigrams as they are more meaningful. In the next few steps I am going to highlight most common bigrams and trigrams for each of 4 categories:

In [240]:
jun_pairs = [ " ".join(pair) for pair in nltk.bigrams(junior_words)]
jun_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(junior_words)])
fdist_junpairs = FreqDist(jun_pairs)
fdist_jun_trigrams = FreqDist(jun_trigrams)


sorted(w for w in set(jun_pairs) if fdist_junpairs[w] > 5)
#sorted(w for w in set(jun_trigrams) if fdist_jun_trigrams[w] > 5)

['Investigate data',
 'analyzing investigating',
 'data analysis',
 'data mining',
 'data models',
 'data parsing',
 'data problems',
 'identify patterns',
 'interpreting modeling',
 'investigating interpreting',
 'love working',
 'mining segmentation',
 'modeling extracting',
 'parsing analyzing',
 'patterns person',
 'person love',
 'problems identify',
 'working data']

In [239]:
mid_low_pairs = [ " ".join(pair) for pair in nltk.bigrams(mid_low_words)]
mid_low_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(mid_low_words)])
fdist_mid_low = FreqDist(mid_low_pairs)
fdist_mid_low_trigrams = FreqDist(mid_low_trigrams)


sorted(w for w in set(mid_low_pairs) if fdist_mid_low[w] > 5)
#sorted(w for w in set(mid_low_trigrams) if fdist_mid_low_trigrams[w] > 2)

['Data Scientist',
 'Physics Division',
 'clinical data',
 'data analysis',
 'data quality',
 'data sets',
 'data sources',
 'data using',
 'ensure data',
 'large data',
 'successful candidate']

In [245]:
mid_high_pairs = [ " ".join(pair) for pair in nltk.bigrams(mid_high_words)]
mid_high_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(mid_high_words)])
fdist_mid_high = FreqDist(mid_high_pairs)
fdist_mid_high_trigrams = FreqDist(mid_high_trigrams)


sorted(w for w in set(mid_high_pairs) if fdist_mid_high[w] > 5)
#sorted(w for w in set(mid_high_trigrams) if fdist_mid_high_trigrams[w] > 2)

['Data Scientist',
 'Experience data',
 'data analysis',
 'data mining',
 'machine learning',
 'mining data',
 'new data',
 'quality data']

In [250]:
senior_pairs = [ " ".join(pair) for pair in nltk.bigrams(senior_words)]
senior_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(senior_words)])
fdist_senior = FreqDist(senior_pairs)
fdist_senior_trigrams = FreqDist(senior_trigrams)


sorted(w for w in set(senior_pairs) if fdist_senior[w] > 3)
#sorted(w for w in set(senior_trigrams) if fdist_senior_trigrams[w] > 2)

['Data Analysis',
 'Data Science',
 'Data Scientist',
 'Data Scientists',
 'Field Data',
 'Scientist experience',
 'data management',
 'data mining',
 'data scientist',
 'data scientists',
 'end end',
 'experience leading',
 'highly motivated',
 'looking highly']

Now it seems that towards senior level words as "expert", "management", "lead" appear more frequently. While words like "parsing" and "extracting" are more common for junior category

In [232]:
lr = sklearn.linear_model.LogisticRegression()
lr.fit(titanic[['Age']] ,titanic.Survived)
probabilities = lr.predict_proba([ [age] for age in range(0,101)  ])
matplotlib.pyplot.plot(probabilities)
probabilities

NameError: name 'sklearn' is not defined